# Pregunta 4

### Lectura

In [1]:
import pandas as pd
import numpy as np
import tarfile
from urllib import urlretrieve

url = 'http://octopus.inf.utfsm.cl/~ricky/movies.tar.gz'

filename, headers = urlretrieve(url, 'movies.tar.gz')

In [2]:
tar = tarfile.open(filename)

dev_x = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/dev.x.mm"
test_x = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/test.x.mm"
dev_y = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/dev.y.dat"
test_y = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/test.y.dat"
train_x = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/train.x.mm"
train_y = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/train.y.dat"
vocab = "movies-preproc/ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/vocab"

dev_x_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/dev.x.mm"
test_x_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/test.x.mm"
dev_y_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/dev.y.dat"
test_y_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/test.y.dat"
train_x_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/train.x.mm"
train_y_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/train.y.dat"
vocab_stars = "movies-preproc/ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/vocab"

for tarinfo in tar:

    if tarinfo.name == dev_x:
        d_x = tar.extractfile(tarinfo.name)
    if tarinfo.name == dev_y:
        d_y = tar.extractfile(tarinfo.name)
    if tarinfo.name == test_x:
        t_x = tar.extractfile(tarinfo.name)
    if tarinfo.name == test_y:
        t_y = tar.extractfile(tarinfo.name)
    if tarinfo.name == train_x:
        tr_x = tar.extractfile(tarinfo.name)
    if tarinfo.name == train_y:
        tr_y = tar.extractfile(tarinfo.name)
    if tarinfo.name == vocab:
        v = tar.extractfile(tarinfo.name)
        
    if tarinfo.name == dev_x_stars:
        d_x_s = tar.extractfile(tarinfo.name)
    if tarinfo.name == dev_y_stars:
        d_y_s = tar.extractfile(tarinfo.name)
    if tarinfo.name == test_x_stars:
        t_x_s = tar.extractfile(tarinfo.name)
    if tarinfo.name == test_y_stars:
        t_y_s = tar.extractfile(tarinfo.name)
    if tarinfo.name == train_x_stars:
        tr_x_s = tar.extractfile(tarinfo.name)    
    if tarinfo.name == train_y_stars:
        tr_y_s = tar.extractfile(tarinfo.name)
    if tarinfo.name == vocab_stars:
        v_s = tar.extractfile(tarinfo.name)

import scipy.io as sio
from scipy.sparse import csr_matrix


v_array = v.read().split("\n")
v_s_array = v_s.read().split("\n")

d_y_array = np.loadtxt(d_y)
t_y_array = np.loadtxt(t_y)
tr_y_array = np.loadtxt(tr_y)
d_y_s_array = np.loadtxt(d_y_s)
t_y_s_array = np.loadtxt(t_y_s)
tr_y_s_array = np.loadtxt(tr_y_s)

d_x_matrix = csr_matrix(sio.mmread(d_x))
t_x_matrix = csr_matrix(sio.mmread(t_x))
tr_x_matrix = csr_matrix(sio.mmread(tr_x))

d_x_s_matrix = csr_matrix(sio.mmread(d_x_s))
t_x_s_matrix = csr_matrix(sio.mmread(t_x_s))
tr_x_s_matrix = csr_matrix(sio.mmread(tr_x_s))

tar.close()

In [11]:
def isMeta(inputString):
    number = any(char.isdigit() for char in inputString)
    return number

def isI(inputString):
    upper = any(char.isupper() for char in inputString)
    lower = any(char.islower() for char in inputString)
    number = any(char.isdigit() for char in inputString)
    if upper == False and lower == True and number == False:
        return True
    else:
        return False

def isII(inputString):
    upper = any(char.isupper() for char in inputString)
    pato = any(char == "<" for char in inputString)
    number = any(char.isdigit() for char in inputString)
    if upper == True and pato == False and number == False:
        return True
    else:
        return False
def isIII(inputString):
    upper = any(char.isupper() for char in inputString)
    pato = any(char == "<" for char in inputString)
    number = any(char.isdigit() for char in inputString)
    if upper == True and pato == True and number == False:
        return True
    else:
        return False

meta = []
I = []
II = []
III = []

count = 0
for i in v_array:
    if isMeta(i):
        meta.append(count)
    elif isI(i):
        I.append(count)
    elif isII(i):
        II.append(count)
    elif isIII(i):
        III.append(count)
    count += 1
    
meta_s = []
I_s = []
II_s = []
III_s = []

count = 0
for i in v_s_array:
    if isMeta(i):
        meta_s.append(count)
    elif isI(i):
        I_s.append(count)
    elif isII(i):
        II_s.append(count)
    elif isIII(i):
        III_s.append(count)
    count += 1
    

Es importante mantener el formato __sparse__ de las matrices, por un tema de optimización de memoria. Es necesario tener solo en memoria los datos no nulos de la matriz. Esto es significativo cuando se trabaja con matrices muy grandes, y donde muchos de los elementos son nulos. Asi, con un formato __sparse__, no hay necesidad de desperdiciar memoria en datos que no estan, haciendo mas eficiente el procesamiento de estas matrices.

### Modelo a utilizar
En esta ocación, el modelo a utilizar para modelar este problema es el _ElasticNet_, propuesto en el paper _"Movie Reviews and Revenues: An Experiment in Text Regression"_. Es una regresion lineal que combina las penalizaciones L1 y L2. Para un $\alpha = 0$, equivale a utilizar _Ridge_ y $\alpha = 1$ a _Lasso_. 

$$\theta = argmin \frac{1}{2n} \sum_{i=1}^{n}{(y_i-(\beta_0 + x^T_i\beta) )^2 + \lambda P(\beta)}$$
$$P(\beta) = \sum_{j=1}^p (\frac{1}{2} (1-\alpha) \beta_j^2+\alpha |\beta_j|)$$

Para estimar los mejores parametros, utilizando los datos de entrenamiento, se fueron variando los valores de $\alpha$ y $\lambda$, evaluando la funcion sobre los datos de desarrollo. A medida que se consige un mejor coeficiente de ajuste, se van guardando los paramentos.


In [46]:
from sklearn.linear_model import ElasticNet

def best_params_dev(b):
    alfa = np.logspace(-2, 1, num=45, base=10)
    model = ElasticNet(fit_intercept = True)
    best_a = 0
    best_b = 0
    best_coef = 0

    for a in alfa:
        model.set_params(alpha = a, l1_ratio = b)
        model.fit(tr_x_s_matrix[:, meta_s], tr_y_s_array)
        coef = model.score(d_x_s_matrix[:,meta_s], d_y_s_array)
        print '{:^15} {:^10} {:^15}'.format(a, b, coef)
        if best_coef < coef:
            best_coef = coef
            best_a = a
            best_b = b
    print "Para b: {0}, el mejor a: {1} con un coeficiente de {2}".format(best_b, best_a, best_coef)        
    
    return a,b

In [47]:
a0,b0 = best_params_dev(0)

     0.01           0      0.586844190516 
0.0116998910659     0      0.586290261877 
0.0136887450954     0      0.585504407298 
0.0160156826445     0      0.584510385322 
0.0187381742286     0       0.58333566997 
0.0219234597248     0      0.582009191968 
0.0256502090568     0       0.58055900975 
0.0300104651782     0      0.579010193249 
0.0351119173422     0      0.577383152772 
0.0410805608018     0      0.575692566163 
0.0480638086306     0      0.573946969494 
0.056234132519      0       0.57214899714 
0.0657933224658     0      0.570296196818 
0.0769774705712     0      0.568382306101 
0.0900628020211     0      0.566398854724 
0.105372497274      0       0.56433694502 
0.123284673944      0      0.562189055971 
0.144241725524      0      0.559950713696 
0.168761247579      0      0.557621875446 
0.197448821282      0      0.555207890649 
0.231012970008      0      0.552719935554 
0.270282658391      0      0.550174868852 
0.316227766017      0      0.547594520141 
0.369983041

In [48]:
a1,b1 = best_params_dev(0.2)

     0.01          0.2      0.58719822283 
0.0116998910659    0.2     0.587003522209 
0.0136887450954    0.2     0.586553125305 
0.0160156826445    0.2     0.585862382223 
0.0187381742286    0.2     0.584952887175 
0.0219234597248    0.2     0.583850809875 
0.0256502090568    0.2     0.582584753782 
0.0300104651782    0.2     0.581183421167 
0.0351119173422    0.2     0.579673374931 
0.0410805608018    0.2     0.578077156565 
0.0480638086306    0.2     0.576411947977 
0.056234132519     0.2     0.574688880154 
0.0657933224658    0.2     0.572913005732 
0.0769774705712    0.2     0.571083883801 
0.0900628020211    0.2     0.569196678439 
0.105372497274     0.2     0.567243637683 
0.123284673944     0.2      0.56521582258 
0.144241725524     0.2      0.56310491729 
0.168761247579     0.2     0.560904975354 
0.197448821282     0.2     0.558613942262 
0.231012970008     0.2     0.556234810658 
0.270282658391     0.2     0.553776285203 
0.316227766017     0.2     0.551252885476 
0.369983041

In [49]:
a2,b2 = best_params_dev(0.4) 

     0.01          0.4     0.586869624749 
0.0116998910659    0.4     0.587159206125 
0.0136887450954    0.4     0.587183871101 
0.0160156826445    0.4     0.586945555173 
0.0187381742286    0.4     0.586453628138 
0.0219234597248    0.4      0.58572457548 
0.0256502090568    0.4     0.584780890284 
0.0300104651782    0.4     0.583649308952 
0.0351119173422    0.4     0.582358614761 
0.0410805608018    0.4     0.580937301174 
0.0480638086306    0.4     0.579411375313 
0.056234132519     0.4      0.57780255731 
0.0657933224658    0.4     0.576127045114 
0.0769774705712    0.4     0.574394935537 
0.0900628020211    0.4     0.572610304547 
0.105372497274     0.4     0.570771885932 
0.123284673944     0.4      0.56887424529 
0.144241725524     0.4     0.566909303844 
0.168761247579     0.4     0.564868101863 
0.197448821282     0.4     0.562742603207 
0.231012970008     0.4     0.560527415637 
0.270282658391     0.4     0.558221261153 
0.316227766017     0.4     0.555828058533 
0.369983041

In [50]:
a3,b3 = best_params_dev(0.6) 

     0.01          0.6     0.584947134696 
0.0116998910659    0.6     0.585884225421 
0.0136887450954    0.6     0.586580479391 
0.0160156826445    0.6     0.587022511726 
0.0187381742286    0.6     0.587201766262 
0.0219234597248    0.6     0.587116021844 
0.0256502090568    0.6     0.586770302478 
0.0300104651782    0.6     0.586177033014 
0.0351119173422    0.6     0.585355382722 
0.0410805608018    0.6     0.584329861953 
0.0480638086306    0.6     0.583128342368 
0.056234132519     0.6     0.581779768468 
0.0657933224658    0.6     0.580311846042 
0.0769774705712    0.6     0.578748981851 
0.0900628020211    0.6     0.577110705312 
0.105372497274     0.6     0.575410699989 
0.123284673944     0.6     0.573656503913 
0.144241725524     0.6     0.571849850026 
0.168761247579     0.6     0.569987570601 
0.197448821282     0.6     0.568062915534 
0.231012970008     0.6     0.566067224919 
0.270282658391     0.6      0.56399170487 
0.316227766017     0.6     0.561829225623 
0.369983041

In [51]:
a4,b4 = best_params_dev(0.8)

     0.01          0.8      0.57835256618 
0.0116998910659    0.8      0.58015781866 
0.0136887450954    0.8     0.581791835557 
0.0160156826445    0.8     0.583241561661 
0.0187381742286    0.8     0.584491221173 
0.0219234597248    0.8     0.585523583666 
0.0256502090568    0.8     0.586321650388 
0.0300104651782    0.8     0.586870565654 
0.0351119173422    0.8     0.587159562336 
0.0410805608018    0.8     0.587183637757 
0.0480638086306    0.8     0.586944744688 
0.056234132519     0.8     0.586452269389 
0.0657933224658    0.8     0.585722711527 
0.0769774705712    0.8     0.584778574914 
0.0900628020211    0.8     0.583646607143 
0.105372497274     0.8     0.582355590292 
0.123284673944     0.8     0.580934016211 
0.144241725524     0.8     0.579407882226 
0.168761247579     0.8      0.5777988985  
0.197448821282     0.8      0.57612324972 
0.231012970008     0.8     0.574391018992 
0.270282658391     0.8     0.572606269453 
0.316227766017     0.8     0.570767724163 
0.369983041

In [52]:
a5,b5 = best_params_dev(1)

     0.01           1      0.434812304941 
0.0116998910659     1      0.434812469641 
0.0136887450954     1      0.434812662336 
0.0160156826445     1      0.434812887403 
0.0187381742286     1      0.434813151006 
0.0219234597248     1      0.434813459522 
0.0256502090568     1      0.434813820479 
0.0300104651782     1       0.43481424279 
0.0351119173422     1      0.434814736884 
0.0410805608018     1      0.434815314962 
0.0480638086306     1      0.434815991296 
0.056234132519      1      0.434816782586 
0.0657933224658     1      0.434817708367 
0.0769774705712     1      0.434818791495 
0.0900628020211     1      0.434820058706 
0.105372497274      1      0.434821541281 
0.123284673944      1      0.434823275809 
0.144241725524      1      0.434825305096 
0.168761247579      1      0.434827679214 
0.197448821282      1      0.434830451368 
0.231012970008      1       0.43483369934 
0.270282658391      1      0.434837499654 
0.316227766017      1      0.434841945537 
0.369983041

Los mejores parametros encontrados fueron:

$\alpha = 0$ y $\lambda = 2.27584592607$

Correlación obtenida : 

In [55]:
model = ElasticNet(fit_intercept = False)
model.set_params(alpha = a3, l1_ratio = b3)
model.fit(tr_x_s_matrix[:, meta_s], tr_y_s_array)

print "R2=%f"%(model.score(t_x_s_matrix[:, meta_s], t_y_s_array))**0.5

R2=0.677757
